In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

This repository contains an educational path search algorithm, just for studying. 

1. **Basic Path Search:** The purpose of basic path search is to find a possible (not necessarily the best) path between two points on a grid cell. If there is at least a path between two poins, this algorithm will find it anyway; however the approach is not efficient. In other words, it always works, but it is not the most efficient way. 
  
2. **A-star Search:** This approach is probably the most efficient way to fond the best path between two points. However, one drawback is that unlike the **basic search** (defined above), it may not be able to find the path in the first forward search and we may need to try different 
 
3. **Dynamic Programming Search:** The purpose of dynamic programming path search is to find all possible and the best directions between two adjucent cells, in order to reach to a specified goal cell. If there is at least a path between two poins, this algorithm will populate all possible directions anyway; if the goal point is blocked by starting init cells, it would result finding no path to the goal point. 

The pipeline consists of two main classes `RouteNode` and `RouteFinder`. 

- Class `RouteNode` encapsulates the posistion of the root node (`pos`), the grid cell indices of the neighbors `neighbor_indices`, and the costs associated with reaching from the root node to its neighbors. 

- Class `RouteFinder` contains the following variables and methods: 
 
  - **Variables**: 
    - `route_nodes`: a list of `RouteNode` objects 
    - `delta_names`: a working (temporary) matrix, with the same size as the `grid_cells`, containing the direction arrows (<, >, ^, v) 
    - `directions`: the same matrix as `delta_names`, but it contains the final direction arrows, extracted from the `delta_names` based on the search approach 
    - `count_table`: an integer matrix, with the same size as the `grid_cells`, containing the number of pipeline iteration while exploring each cell 
    - `search_counter`: a cumulative integer counter, keeping the cumulative number of pipeline iteration while exploring each cell 
    - `cost_table`: this table is a matrix that includes the cost of a path from each cell or node of a grid to the specified goal cell or node. 
        
  - **Main Functions**:  
    - `search_a_star`: the A*-based search algorithm to find a path between two points on the map. 
    - `search_dynamic_programming`: this method is designed based on basic search approach, finding a path between two points on the map. It also bookkeeps the neighbors for cost table calculation. This algorithm does not account for direction of the vehicle and the roads. As long as a path exists between two cells, this fucntion finds an existing path anyway, regardless of the costs of each action (right turn, left turn, u trun, and moving forward). The only cost factor is accounted for in this search algorithm is the move to neighboring cells, i.e. `cost = 1` associated with each action. 
    - `calculate_optimum_policy`: this method find the optimum policy via calculating the `cost_table` matrix. As explained above, `cost_table` matrix includes the cost of a path from each node of a grid to the specified goal node. This method uses `search_dynamic_programming` method in some iterations to calculate the `cost_table`; therefore, the direction of the vehicle is not accounted for while calculating the `cost_table`. The final result of this method is stored in `direction_table`, as a member variable of `RouteFinder` class. 

  - **Supplementary Functions**: 
    - `extract_paths_based_on_optimum_policy`: to summarize and extract the final output out of the search method selected 
    - `extract_route_based_on_optimum_policy`: 


In [2]:
import numpy as np

In [6]:
class RouteNode:
    def __init__(self, states, neighbor_indices, costs):
        """
        @states: the node indices and direction 
        @neighbor_indices: the indices of the accessible neighbors
        @costs: the costs of reaching from the current node to the neighbor nodes or indices 
        """
        
        self.states = states
        self.neighbor_indices = neighbor_indices
        self.costs = costs
        
        # to calculate policy_costs, the vehicle's direction must be taken into account  
        # self.policy_costs = [-1] * len(costs)


In [7]:
class RouteFinder:
    
    __cost_default_value__ = -2
    
    def __init__(self, grid_shape, barriers_init_cost):
        self.route_nodes = []
        # self.direction_table_table = np.array(['apples', 'foobar', 'cowboy'], dtype=object)
        self.delta_names = np.full(grid_shape, fill_value="", dtype=object)
        self.direction_table = np.copy(self.delta_names)
        self.count_table = np.ones(grid_shape) * -1.0
        # self.value_table = np.copy(self.count_table)
        self.search_counter = 0
        self.barriers_init_cost = barriers_init_cost
        
        self.cost_table_counter = 0
        self.cost_table = np.ones(grid_shape) * RouteFinder.__cost_default_value__
        # self.desirable_directions = np.ones(grid_shape) * -1.0
        self.neighbors_indices_table = dict()
    
    def reset(self, grid_shape):
        self.route_nodes = []
        # self.direction_table_table = np.array(['apples', 'foobar', 'cowboy'], dtype=object)
        self.delta_names = np.full(grid_shape, fill_value="", dtype=object)
        self.direction_table = np.copy(self.delta_names)
        self.count_table = np.ones(grid_shape) * -1.0
        # self.value_table = np.copy(self.count_table)
        # self.cost_table = np.ones(grid_shape) * RouteFinder.__cost_default_value__
        # self.desirable_directions = np.ones(grid_shape) * -1.0
        self.search_counter = 0
        
    def reset_cost_table(self, grid_shape):
        self.cost_table_counter = 0
        self.cost_table = np.ones(grid_shape) * RouteFinder.__cost_default_value__
        self.neighbors_indices_table = dict()
    
    @staticmethod
    def get_neighbors_indices(pos, grid_cells):
        max_size_x = grid_cells.shape[0] - 1
        max_size_y = grid_cells.shape[1] - 1

        # pos = [min(current_pos[0], max_size_x), min(current_pos[1], max_size_y)]

        # up
        up_neighbor_indices = [max(pos[0] - 1, 0), pos[1]]
        # down
        down_neighbor_indices = [min(pos[0] + 1, max_size_x), pos[1]]
        # left
        left_neighbor_indices = [pos[0], max(pos[1] - 1, 0)]
        # right
        right_neighbor_indices = [pos[0], min(pos[1] + 1, max_size_y)]
        
        return (up_neighbor_indices, down_neighbor_indices, left_neighbor_indices, right_neighbor_indices)
    
    def extract_neighbors_indices(self, unique_index_id):
        # return (up_neighbor_indices, down_neighbor_indices, left_neighbor_indices, right_neighbor_indices)
        
        return (self.neighbors_indices_table[unique_index_id][0], 
                self.neighbors_indices_table[unique_index_id][1], 
                self.neighbors_indices_table[unique_index_id][2], 
                self.neighbors_indices_table[unique_index_id][3])
    
    def find_neighbor_indices_basic(self, states, grid_cells, modify_grid_cells, turn_costs):
        
        """
        @turn_costs: the costs associated with making a 
            0: right turn
            1: no turn
            2: left turn
        
        @pos: given in the form [row, col, direction]
            direction: 
            0: up
            1: left
            2: down 
            3: right 
        
        Note: When this function is called for the first time, after finding the neighbor indices, it also book-keeps them in a hashtable for future calls 
        """

        neighbors_indices = [[], [], [], []]
        costs = [[], [], [], []]
        pos = [states[0], states[1]]
        direction = states[2]
        
        # up direction 
        if (direction == 0):
            # relative to up, down, left, right
            # no turn, no turn, left turn, right turn
            navigable_turns_indices = [1, 1, 2, 0]
        # left direction 
        elif (direction == 1):
            # relative to up, down, left, right
            # right turn, left turn, no turn, no turn 
            navigable_turns_indices = [0, 2, 1, 1]
        # down direction 
        elif (direction == 2):
            # relative to up, down, left, right
            # no turn, no turn, right turn, left turn
            navigable_turns_indices = [1, 1, 0, 2]
        # right direction 
        elif (direction == 3):
            # relative to up, down, left, right
            # left turn, right turn, no turn, no turn
            navigable_turns_indices = [2, 0, 1, 1]
        else:
            # no preference 
            # turn_costs = [1, 1, 1, 1]
            # navigable_turns_indices = [0, 1, 2, 3]
            raise Exception("invalid direction")
        
        # print("navigable_turns_indices", navigable_turns_indices)
        
        unique_index_id = pos[0] * grid_cells.shape[1] + pos[1]
        
        if (unique_index_id in self.neighbors_indices_table):
            # print("fast:", pos, self.neighbors_indices_table[unique_index_id])
            
            (up_neighbor_indices, down_neighbor_indices, left_neighbor_indices, right_neighbor_indices) = self.extract_neighbors_indices(unique_index_id)
        else:
            (up_neighbor_indices, down_neighbor_indices, left_neighbor_indices, right_neighbor_indices) = RouteFinder.get_neighbors_indices(pos, grid_cells)
            self.neighbors_indices_table[unique_index_id] = [[], [], [], []]
        
        # index = 0
        # routs_count = 0
        
        if (up_neighbor_indices != [] and pos != up_neighbor_indices):
            self.neighbors_indices_table[unique_index_id][0] = [up_neighbor_indices[0], up_neighbor_indices[1]]
            
            if (grid_cells[up_neighbor_indices[0], up_neighbor_indices[1]] == 0):
                if (modify_grid_cells):
                    # self.search_counter += 1
                    grid_cells[up_neighbor_indices[0], up_neighbor_indices[1]] = 1
                    # self.count_table[up_neighbor_indices[0], up_neighbor_indices[1]] = self.search_counter
                    # self.delta_names[up_neighbor_indices[0], up_neighbor_indices[1]] = "^"
                    # self.cost_table[up_neighbor_indices[0], up_neighbor_indices[1]] = 

                # self.neighbors_indices_table[unique_index_id].append([up_neighbor_indices[0], up_neighbor_indices[1]])
                neighbors_indices[0] = [up_neighbor_indices[0], up_neighbor_indices[1]]
                costs[0] = turn_costs[navigable_turns_indices[0]]
                # index += 1

        if (down_neighbor_indices != [] and up_neighbor_indices != down_neighbor_indices) and (pos != down_neighbor_indices):
            self.neighbors_indices_table[unique_index_id][1] = [down_neighbor_indices[0], down_neighbor_indices[1]]
            
            if (grid_cells[down_neighbor_indices[0], down_neighbor_indices[1]] == 0): 
                if (modify_grid_cells):
                    grid_cells[down_neighbor_indices[0], down_neighbor_indices[1]] = 1

                # self.neighbors_indices_table[unique_index_id].append([down_neighbor_indices[0], down_neighbor_indices[1]])
                neighbors_indices[1] = [down_neighbor_indices[0], down_neighbor_indices[1]]
                costs[1] = turn_costs[navigable_turns_indices[1]]
                # index += 1

        if (left_neighbor_indices != [] and pos != left_neighbor_indices):
            self.neighbors_indices_table[unique_index_id][2] = [left_neighbor_indices[0], left_neighbor_indices[1]]
            
            if (grid_cells[left_neighbor_indices[0], left_neighbor_indices[1]] == 0):
                if (modify_grid_cells):
                    grid_cells[left_neighbor_indices[0], left_neighbor_indices[1]] = 1

                # self.neighbors_indices_table[unique_index_id].append([left_neighbor_indices[0], left_neighbor_indices[1]])
                neighbors_indices[2] = [left_neighbor_indices[0], left_neighbor_indices[1]]
                costs[2] = turn_costs[navigable_turns_indices[2]]
                # index += 1

        if (right_neighbor_indices != [] and left_neighbor_indices != right_neighbor_indices) and (pos != right_neighbor_indices):
            self.neighbors_indices_table[unique_index_id][3] = [right_neighbor_indices[0], right_neighbor_indices[1]]
            
            if (grid_cells[right_neighbor_indices[0], right_neighbor_indices[1]] == 0):
                if (modify_grid_cells):
                    grid_cells[right_neighbor_indices[0], right_neighbor_indices[1]] = 1

                # self.neighbors_indices_table[unique_index_id].append([right_neighbor_indices[0], right_neighbor_indices[1]])
                neighbors_indices[3] = [right_neighbor_indices[0], right_neighbor_indices[1]]
                costs[3]  = turn_costs[navigable_turns_indices[3]]
                # index += 1

        return (neighbors_indices, costs)
    
    def find_neighbor_pos_astar(self, pos, grid_cells, heuristic_cells, modify_grid_cells):
    
        # neighbor_poses = []

        neighbor_indices = []
        neighbor_pos = []
        delta_names = [" "] * 4
        # costs = []
        cost = -1
        astar_costs = np.ones([4]) * 100

        (up_neighbor_indices, down_neighbor_indices, left_neighbor_indices, right_neighbor_indices) = RouteFinder.get_neighbors_indices(pos, grid_cells)
        
        # print("up_neighbor_indices", up_neighbor_indices)
        # print("down_neighbor_indices", down_neighbor_indices)
        # print("left_neighbor_indices", left_neighbor_indices)
        # print("right_neighbor_indices", right_neighbor_indices)

        index = 0

        if ( (pos != up_neighbor_indices) and 
             (grid_cells[up_neighbor_indices[0], up_neighbor_indices[1]] == 0)):
            
            astar_costs[index] = heuristic_cells[up_neighbor_indices[0], up_neighbor_indices[1]] + self.search_counter
            delta_names[index] = "^"
            neighbor_indices.append([up_neighbor_indices[0], up_neighbor_indices[1]])
            index += 1

        if ( (up_neighbor_indices != down_neighbor_indices) and (pos != down_neighbor_indices) and 
             (grid_cells[down_neighbor_indices[0], down_neighbor_indices[1]] == 0)):

            astar_costs[index] = heuristic_cells[down_neighbor_indices[0], down_neighbor_indices[1]] + self.search_counter
            delta_names[index] = "v"
            neighbor_indices.append([down_neighbor_indices[0], down_neighbor_indices[1]])
            index += 1
            
            # print("down:", [down_neighbor_indices[0], down_neighbor_indices[1]])

        if ( (pos != left_neighbor_indices) and 
             (grid_cells[left_neighbor_indices[0], left_neighbor_indices[1]] == 0)):

            astar_costs[index] = heuristic_cells[left_neighbor_indices[0], left_neighbor_indices[1]] + self.search_counter
            delta_names[index] = "<"
            neighbor_indices.append([left_neighbor_indices[0], left_neighbor_indices[1]])
            index += 1
            
            # print("left:", [left_neighbor_indices[0], left_neighbor_indices[1]])

        if ( (left_neighbor_indices != right_neighbor_indices) and (pos != right_neighbor_indices) and 
             (grid_cells[right_neighbor_indices[0], right_neighbor_indices[1]] == 0)):

            astar_costs[index] = heuristic_cells[right_neighbor_indices[0], right_neighbor_indices[1]] + self.search_counter
            delta_names[index] = ">"
            neighbor_indices.append([right_neighbor_indices[0], right_neighbor_indices[1]])
            index += 1

            # print("right:", [right_neighbor_indices[0], right_neighbor_indices[1]])
            
        if (index > 0):
            sorted_indices = np.argsort(astar_costs)
            
            selected_pos = [neighbor_indices[sorted_indices[0]][0], neighbor_indices[sorted_indices[0]][1]]
            
            if (modify_grid_cells):
                grid_cells[selected_pos[0], selected_pos[1]] = 1
                self.count_table[selected_pos[0], selected_pos[1]] = self.search_counter
                self.delta_names[selected_pos[0], selected_pos[1]] = delta_names[sorted_indices[0]]
                self.search_counter += 1

            neighbor_pos = [selected_pos[0], selected_pos[1]]
            cost = 1.0
            
            # print("astar_costs:", astar_costs, "  -  selected_pos:", selected_pos, "  -  neighbor_pos:", neighbor_pos)
            
        return (neighbor_pos, cost)
    
    def extract_path_indices(self, init_pos, goal_pos):
        
        pointer_pos = goal_pos.copy()
        path_cell_indices = [pointer_pos]
        # path_delta_names = []
        total_cost = 0.0
        node_index = len(self.route_nodes) - 1
        
        while (pointer_pos != init_pos):
            if (node_index < 0):
                return ([], [], 0.0)
            
            node = self.route_nodes[node_index]
            
            if (pointer_pos in node.neighbor_indices):
                index = node.neighbor_indices.index(pointer_pos)
                path_cell_indices.append(node.pos)
                # path_delta_names.append(node.delta_names[index])
                total_cost += node.costs[index]
                pointer_pos = node.pos
                
                # print(node.pos)
            
            node_index -= 1
        
        path_cell_indices.reverse()
        # path_delta_names.reverse()
        
        return (path_cell_indices, total_cost)
    
    def extract_paths_based_on_optimum_policy(self, init_states, goal_pos, grid_shape):

        init_pos = [init_states[0], init_states[1]]
        pointer_pos = goal_pos.copy()
        # path_delta_names = []
        total_cost = 0.0
        node_index = len(self.route_nodes) - 1
        
        while (pointer_pos != init_pos):
            if (node_index < 0):
                return ([], [], 0.0)
            
            node = self.route_nodes[node_index]
            
            if (pointer_pos in node.neighbor_indices):
                index = node.neighbor_indices.index(pointer_pos)
                total_cost += node.costs[index]
                node_pos = [node.states[0], node.states[1]]
                self.direction_table[node_pos[0], node_pos[1]] = self.delta_names[pointer_pos[0], pointer_pos[1]]
                self.cost_table[node_pos[0], node_pos[1]] = total_cost
                
                # search_key = [key for (key, value) in aa.items() if value == [node.pos[0], node.pos[1]]]
                
                # print("self.cost_table", node.pos, self.cost_table[node.pos[0], node.pos[1]])
                pointer_pos = node_pos
            
            node_index -= 1
        
        return total_cost
    
    def search_dynamic_programming(self, grid_cells, init_states, goal_pos, max_depth, cell_blockage_value):
        
        init_pos = [init_states[0], init_states[1]]
        
        if (init_pos == goal_pos):
            return [0.0, goal_pos[0], goal_pos[1]]
        
        if (grid_cells[init_pos[0], init_pos[1]] == 1):
            return [cell_blockage_value, goal_pos[0], goal_pos[1]]
        
        turn_costs = [1, 1, 1, 1]
        self.reset(grid_cells.shape)
        
        current_pos = init_pos
        current_states = init_states

        branch_search_completed = False
        debug_counter = 0

        grid_cells[init_pos[0], init_pos[1]] = 1
        
        # self.search_counter = 0
        # self.count_table[init_pos[0], init_pos[1]] = 0
        
        (neighbor_indices, costs) = self.find_neighbor_indices_basic(current_states, grid_cells, True, turn_costs)
        neighbor_indices = list(filter(lambda item: item != [], neighbor_indices))
        costs = list(filter(lambda item: item != [], costs))

        # for debug
        # print("current_pos: ", current_pos, "  -  neighbor_indices: ", neighbor_indices)
        new_nodes = []
        current_node = RouteNode(current_states, neighbor_indices, costs)
        new_nodes.append(current_node)
        self.route_nodes += new_nodes
        
        while (not branch_search_completed):
            # index = 0

            neighbor_nodes = []

            for node in new_nodes:

                for node_pos in node.neighbor_indices:
                    # if (node_pos == []):
                    #    continue
                    node_states = [node_pos[0], node_pos[1], init_states[2]]
                    (neighbor_indices, costs) = self.find_neighbor_indices_basic(node_states, grid_cells, True, turn_costs)
                    neighbor_indices = list(filter(lambda item: item != [], neighbor_indices))
                    costs = list(filter(lambda item: item != [], costs))
                    
                    # for debug
                    # print("=================================================")
                    # print("node_pos: ", node_pos, "  -  neighbor_indices: ", neighbor_indices)
                    
                    current_node = RouteNode(node_states, neighbor_indices, costs)
                    neighbor_nodes.append(current_node)
                    
                    for neighbor_pos in neighbor_indices:
                        # if (node_pos == []):
                        #    continue
                        # print("Hamid - 1", neighbor_pos)

                        if (neighbor_pos == goal_pos):
                            branch_search_completed = True

                            self.route_nodes += neighbor_nodes
                            total_cost = self.extract_paths_based_on_optimum_policy(init_pos, goal_pos, grid_cells.shape)
                            
                            # print("found", neighbor_pos)

                            return [total_cost, int(goal_pos[0]), int(goal_pos[1])]

            new_nodes = neighbor_nodes
            self.route_nodes += new_nodes
            
            if (debug_counter == max_depth):
                branch_search_completed = True
            
            debug_counter += 1
        
        return [cell_blockage_value, goal_pos[0], goal_pos[1]]
    
    def search_a_star(self, grid_cells, heuristic_cells, init_pos, goal_pos, max_depth):
        
        if (init_pos == goal_pos):
            return [0.0, goal_pos[0], goal_pos[1]]
        
        if (grid_cells[init_pos[0], init_pos[1]] == 1):
            return [-1.0, goal_pos[0], goal_pos[1]]
        
        self.reset(grid_cells.shape)
        
        current_pos = init_pos

        branch_search_completed = False
        debug_counter = 0

        grid_cells[init_pos[0], init_pos[1]] = 1
        
        self.search_counter = 1
        self.count_table[init_pos[0], init_pos[1]] = 0

        (neighbor_pos, cost) = self.find_neighbor_pos_astar(current_pos, grid_cells, heuristic_cells, modify_grid_cells = True)
        # print for debug
        # print("current_pos: ", current_pos, "  -  neighbor_pos: ", neighbor_pos)
        new_nodes = []
        current_node = RouteNode(current_pos, [neighbor_pos], [cost])
        new_nodes.append(current_node)
        self.route_nodes += new_nodes

        while (not branch_search_completed):
            # index = 0

            neighbor_nodes = []

            for node in new_nodes:

                if (neighbor_pos == goal_pos):
                    branch_search_completed = True

                    self.route_nodes += neighbor_nodes

                    total_cost = self.extract_paths_based_on_optimum_policy(init_pos, goal_pos, grid_cells.shape)

                    return [total_cost, int(goal_pos[0]), int(goal_pos[1])]
                      
                # print for debug
                # print("=================================================")
                # print("node_pos: ", node.neighbor_indices[0])
                
                # for node_pos in node.neighbor_indices:
                (neighbor_pos, cost) = self.find_neighbor_pos_astar(node.neighbor_indices[0], grid_cells, heuristic_cells, modify_grid_cells = True)

                # print for debug
                # print("neighbor_pos: ", neighbor_pos)

                if (neighbor_pos == []):
                    branch_search_completed = True
                    break
                
                current_node = RouteNode(node.neighbor_indices[0], [neighbor_pos], [cost])
                neighbor_nodes.append(current_node)

            new_nodes = neighbor_nodes
            self.route_nodes += new_nodes
            
            if (debug_counter == max_depth):
                branch_search_completed = True
            
            debug_counter += 1
        
        return [-1.0, goal_pos[0], goal_pos[1]]
    
    def calculate_optimum_policy(self, grid_cells, goal_pos, cell_blockage_value):
        # self.value_table = np.ones(grid_cells.shape)
        # self.cost_table = np.ones_like(grid_cells) * RouteFinder.__cost_default_value__
        self.reset_cost_table(grid_cells.shape)
        
        self.cost_table[grid_cells == 1] = -1
        self.cost_table[goal_pos[0], goal_pos[1]] = 0
        
        all_costs_are_estimated = False
        prev_init_pos = []
        init_pos = [-1, -1]
        
        depth_counter = 0
        
        # in case the route to goal is blocked 
        # max_fail_count = 10
        # pointer_counter = 0
        
        while (True):
            remaining_indices = np.argwhere(self.cost_table == RouteFinder.__cost_default_value__)
                        
            all_costs_are_estimated = remaining_indices.size == 0
            # print("remaining_indices", remaining_indices)
            # print("self.cost_table", self.cost_table)
                        
            # if (all_costs_are_estimated or prev_init_pos == init_pos or depth_counter == 10):
            
            if (all_costs_are_estimated or prev_init_pos == init_pos):
                self.cost_table[self.cost_table == -1] = cell_blockage_value
                break
                        
            depth_counter += 1
            
            # if (depth_counter > 10):
            #    print("debug counter limit is reached")
            #    break
            
            copy_grid_cells = np.copy(grid_cells)
            prev_init_pos = init_pos
            # using up direction as the default, but the algorithm does not account for directionality 
            init_pos = [remaining_indices[0][0], remaining_indices[0][1], 0]
            
            search_results = self.search_dynamic_programming(copy_grid_cells, init_pos, goal_pos, max_depth = 100, 
                                                             cell_blockage_value = cell_blockage_value)
        
        # print(self.cost_table)
        
        navigable_indices = np.argwhere(np.logical_and(self.cost_table != cell_blockage_value, self.cost_table != RouteFinder.__cost_default_value__)).tolist()
        navigable_indices.remove(goal_pos)
                
        neighbor_indices = [[], [], [], []]
        
        unique_index_id = goal_pos[0] * grid_cells.shape[1] + goal_pos[1]
        self.neighbors_indices_table[unique_index_id] = [[], [], [], []]
        delta_names = ["^", "v", "<", ">"]
        
        self.direction_table[goal_pos[0], goal_pos[1]] = "*"
        
        # print(cost_table_indices)
        
        for pos in navigable_indices:
            unique_index_id = pos[0] * grid_cells.shape[1] + pos[1]
            
            # print(pos, unique_index_id)
            # (up_neighbor_indices, down_neighbor_indices, left_neighbor_indices, right_neighbor_indices) = route_finder.extract_neighbors_indices(unique_index_id)
            (neighbor_indices[0], neighbor_indices[1], neighbor_indices[2], neighbor_indices[3]) = self.extract_neighbors_indices(unique_index_id)
            costs = [self.cost_table[neighbor_pos[0], neighbor_pos[1]] if neighbor_pos != [] else 1.0e50 for neighbor_pos in neighbor_indices]
            
            index_min = np.argmin(costs)
            # print(pos, costs, index_min)
            self.direction_table[pos[0], pos[1]] = delta_names[index_min]



### A Few Examples 

Use or add to one of these sample grid cells 

In [8]:
# Grid format: 
#   0 = Navigable space 
#   1 = Occupied space 

grid_cells_01 = np.asarray( 
        [[0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 1, 1, 1, 0],
         [0, 0, 0, 0, 1, 0]] )

grid_cells_02 = np.asarray( 
        [[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 1, 1, 1, 0],
        [0, 0, 1, 0, 1, 0]] )

grid_cells_03 = np.asanyarray(
    [[0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 0]] )

grid_cells_04 = np.asarray(
    [[0, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 0]] )

grid_cells_05 = np.asarray(
    [[0, 0, 1, 0, 0, 0],
     [0, 0, 1, 0, 0, 0],
     [0, 0, 1, 0, 1, 0],
     [0, 0, 1, 1, 1, 0],
     [0, 0, 0, 0, 1, 0]] )

grid_cells_06 = np.asarray(
    [[0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 0]] )

grid_cells_07 = np.asarray(
    [[1, 1, 1, 0, 0, 0],
     [1, 1, 1, 0, 1, 0],
     [0, 0, 0, 0, 0, 0],
     [1, 1, 1, 0, 1, 1],
     [1, 1, 1, 0, 1, 1]] )

heuristic_cells = np.asarray(
    [[9, 8, 7, 6, 5, 4],
     [8, 7, 6, 5, 4, 3],
     [7, 6, 5, 4, 3, 2],
     [6, 5, 4, 3, 2, 1],
     [5, 4, 3, 2, 1, 0]] )


forward_moves = [[-1, 0], # go up
                 [ 0,-1], # go left
                 [ 1, 0], # go down
                 [ 0, 1]] # go right

forward_names = ["up", "left", "down", "right"]


In [9]:

# turn_costs: making a right turn, no turn, and a left turn
# turn_costs: the costs associated with making a right turn, left turn, U-turn, no turn

turn_costs = [2, 20, 100, 1]
turn_costs = [2, 20, 100, 1]
# actions: right turn, no turn, left turn
actions = [-1, 0, 1]
action_names = ["R", "#", "L"]

# given in the form [row, col, direction]
# directions: 
# 0: up
# 1: left
# 2: down
# 3: right

init_states = [4, 3, 0]
# init_pos = [0, 0]
# init_pos = [4, 3]

# given in the form [row, col]
# original_goal_pos = [len(grid_cells)-1, len(grid_cells[0])-1]
original_goal_pos = [4, 5]
original_goal_pos = [2, 0]

grid_cells = np.copy(grid_cells_07)
route_finder = RouteFinder(grid_cells.shape, barriers_init_cost=100)

"""
print("========== A* Search: start ==========")
print("grid Cell:\n", grid_cells)
search_results = route_finder.search("a-star", grid_cells, heuristic_cells, init_pos, original_goal_pos, max_depth = 100)
print(search_results)
print("count_table a-star:\n", route_finder.count_table)
print("directions a-star:\n", route_finder.direction_table)
print("==================== End ====================")
"""

print("")

grid_cells = np.copy(grid_cells_07)

print("========== Dynamic Programming Search: start ==========")
print("grid Cell:\n", grid_cells)
search_results = route_finder.calculate_optimum_policy(grid_cells, original_goal_pos, cell_blockage_value = 1000)
print("cost_table:\n", route_finder.cost_table)
print("the result of optimum_policy (direction_table):\n", route_finder.direction_table)
# print("value_table:\n", route_finder.value_table)
print("==================== End ====================")



========== Dynamic Programming Search: start ==========
grid Cell:
 [[1 1 1 0 0 0]
 [1 1 1 0 1 0]
 [0 0 0 0 0 0]
 [1 1 1 0 1 1]
 [1 1 1 0 1 1]]
cost_table:
 [[1000. 1000. 1000.    5.    6.    7.]
 [1000. 1000. 1000.    4. 1000.    6.]
 [   0.    1.    2.    3.    4.    5.]
 [1000. 1000. 1000.    4. 1000. 1000.]
 [1000. 1000. 1000.    5. 1000. 1000.]]
the result of optimum_policy (direction_table):
 [['' '' '' 'v' '<' 'v']
 ['' '' '' 'v' '' 'v']
 ['*' '<' '<' '<' '<' '<']
 ['' '' '' '^' '' '']
 ['' '' '' '^' '' '']]
==================== End ====================
